### notebook 경로 변경

In [2]:
import os
import sys
print(f"현재 노트북 경로: {os.getcwd()}")
# os.chdir("원하는 노트북 경로 설정...")
print(f"파이썬 라이브러리 인식 경로: {sys.path}")

현재 노트북 경로: /Users/admin/workspace/python/DE
파이썬 라이브러리 인식 경로: ['/Users/admin/workspace/python/DE', '/Users/admin/.pyenv/versions/3.10.12/lib/python310.zip', '/Users/admin/.pyenv/versions/3.10.12/lib/python3.10', '/Users/admin/.pyenv/versions/3.10.12/lib/python3.10/lib-dynload', '', '/Users/admin/workspace/python/DE/.venv/lib/python3.10/site-packages']


In [40]:
import sqlite3
from missions.W1.M3.log.log import Logger

logger = Logger.get_logger("AnalysisSQL")

### GDP가 100B USD 이상이 되는 국가만 출력하기

In [41]:
with sqlite3.connect("missions/W1/M3/data/World_Economies.db")  as conn:
    cursor = conn.cursor()
    cursor.execute(
        '''
        SELECT * FROM gdp
        WHERE GDP_USD_billion >= 100
        ORDER BY GDP_USD_billion DESC
        '''
        # IS NOT NULL을 하지 않아도 됨.
        )
    rows = cursor.fetchall()
    logger.info(f"GDP가 100B USD이상이 되는 국가 수: {len(rows)}", )
    for row in rows:
        logger.info(row)

[2025-01-09-17-57-52], GDP가 100B USD이상이 되는 국가 수: 72
[2025-01-09-17-57-52], (1, 'United States', 'North America', 30337.16, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (2, 'China', 'East Asia', 19534.89, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (3, 'Germany', 'Western Europe', 4921.56, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (4, 'Japan', 'East Asia', 4389.33, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (5, 'India', 'South Asia', 4271.92, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (6, 'United Kingdom', 'Western Europe', 3730.26, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (7, 'France', 'Western Europe', 3283.43, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (8, 'Italy', 'Western Europe', 2459.6, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (9, 'Canada', 'North America', 2330.31, 2025, 'IMF', '2025-01-09 08:57:45')
[2025-01-09-17-57-52], (10, 'Brazil', 'Latin Americ

### 각 Region별 top5 국가의 GDP 평균 구하기

In [44]:
# 국가 이름과 각 국가별 GDP까지 나오는 코드
"""
WITH RankedGDP AS (
            SELECT 
                Country,
                Region,
                GDP_USD_billion,
                ROW_NUMBER() OVER (
                    PARTITION BY Region 
                    ORDER BY GDP_USD_billion DESC
                ) as rank
            FROM gdp
        )
        SELECT 
            Region,
            Country,
            ROUND(GDP_USD_billion, 2) as GDP_USD_billion,
            rank,
            ROUND((
                SELECT AVG(GDP_USD_billion)
                FROM RankedGDP r2
                WHERE r2.Region = r1.Region
                AND r2.rank <= 5
            ), 2) as region_top5_avg_GDP
        FROM RankedGDP r1
        WHERE rank <= 5
        ORDER BY 
            region_top5_avg_GDP DESC,
            Region,
            rank
        ;
"""           

with sqlite3.connect("missions/W1/M3/data/World_Economies.db")  as conn:
    cursor = conn.cursor()
    cursor.execute(
        '''
        SELECT 
            Region,
            ROUND(avg(GDP_USD_billion), 2) as top_5_avg_GDP
            /* 반올림을 안하면 파이썬 차원에서 또 다시 소숫점 둘째 자리 뒤까지 보여준다. */
        FROM (
            SELECT
                Country,
                Region,
                GDP_USD_billion,
                ROW_NUMBER() OVER (
                    PARTITION BY Region
                    ORDER BY GDP_USD_billion DESC
                    ) as rank
                    FROM gdp
                )
        WHERE rank <= 5
        GROUP BY Region
        ORDER BY top_5_avg_GDP DESC
        ;

        '''
        # IS NOT NULL을 하지 않아도 됨.
        )
    rows = cursor.fetchall()
    logger.info(f"결과 수: {len(rows)}", )
    for row in rows:
        logger.info(row)


[2025-01-09-18-23-06], 결과 수: 12
[2025-01-09-18-23-06], ('North America', 8622.34)
[2025-01-09-18-23-06], ('East Asia', 5421.57)
[2025-01-09-18-23-06], ('Western Europe', 3244.49)
[2025-01-09-18-23-06], ('Eastern Europe', 1085.0)
[2025-01-09-18-23-06], ('South Asia', 1044.18)
[2025-01-09-18-23-06], ('Latin America', 797.57)
[2025-01-09-18-23-06], ('Southeast Asia', 711.89)
[2025-01-09-18-23-06], ('Middle East', 598.13)
[2025-01-09-18-23-06], ('Oceania', 436.66)
[2025-01-09-18-23-06], ('Sub-Saharan Africa', 215.48)
[2025-01-09-18-23-06], ('North Africa', 182.07)
[2025-01-09-18-23-06], ('Central Asia', 108.96)
